In [ ]:
%load_ext google.cloud.bigquery
import os
import pandas as pd
import numpy as np
import subprocess

In [ ]:
#skeleton code below is used to read csv's that have been written to workspace storage  
 # Get ehr data from workspace bucket
name_of_file_in_bucket = 'colitis_updated.csv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

# read in csv
colitis = pd.read_csv(name_of_file_in_bucket)

In [ ]:
#########reading in colitis + cancer table
name_of_file_in_bucket = 'crc_and_colitis_pts.csv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

# read in csv
crc_and_colitis = pd.read_csv(name_of_file_in_bucket)

In [ ]:
#pull data in 'person', 'concept', and 'survey' tables 

####add indicator column for outcome
colitis['crc_yes'] = 0

colitis.loc[colitis['person_id'].isin(crc_and_colitis['person_id']), 'crc_yes'] = 1

len(colitis[colitis['crc_yes'] == 1]['person_id'].unique()) #sanity check, should equal 168

In [ ]:
dataset = os.getenv("WORKSPACE_CDR")
CDR_split=dataset.split(".")
CDR_version=CDR_split[1]
prefix = CDR_split[0]

#prefix
CDR_version

In [ ]:
###testing SQL syntax
test_query = '''
SELECT *
FROM fc-aou-cdr-prod-ct.C2022Q4R9.observation
WHERE value_as_string != 'None'
LIMIT 1000
'''

test_df = pd.read_gbq(test_query, dialect="standard")
test_df

In [ ]:
#####get demographics
ids_list = colitis['person_id'].unique()
person_query = '''
SELECT 
        person.person_id,
        g_concept.concept_name as gender, 
        person.birth_datetime as date_of_birth,
        r_concept.concept_name as race,
        e_concept.concept_name as ethnicity,
        s_concept.concept_name as birth_sex
    FROM
        fc-aou-cdr-prod-ct.C2022Q4R9.person as person
    LEFT JOIN
        (SELECT concept_id, concept_name FROM fc-aou-cdr-prod-ct.C2022Q4R9.concept) as g_concept
        ON person.gender_concept_id = g_concept.concept_id 
    LEFT JOIN 
        (SELECT concept_id, concept_name FROM fc-aou-cdr-prod-ct.C2022Q4R9.concept) as r_concept
        ON person.race_concept_id = r_concept.concept_id 
    LEFT JOIN 
        (SELECT concept_id, concept_name FROM fc-aou-cdr-prod-ct.C2022Q4R9.concept) as e_concept
        ON person.ethnicity_concept_id = e_concept.concept_id
    LEFT JOIN 
        (SELECT concept_id, concept_name FROM fc-aou-cdr-prod-ct.C2022Q4R9.concept) as s_concept
        ON person.sex_at_birth_concept_id = s_concept.concept_id
    WHERE 
        person.person_id IN ({})
        '''.format(', '.join(map(str, ids_list)))

demo_df = pd.read_gbq(person_query, dialect="standard")
demo_df

In [ ]:
#####get survey data
survey_query='''
SELECT
        ds_survey.person_id,
        ds_survey.survey_datetime,
        ds_survey.survey,
        ds_survey.question_concept_id,
        ds_survey.question,
        ds_survey.answer_concept_id,
        ds_survey.answer,
FROM fc-aou-cdr-prod-ct.C2022Q4R9.ds_survey as ds_survey
WHERE
        (
            question_concept_id IN (
                SELECT
                    DISTINCT concept_id 
                FROM
                    fc-aou-cdr-prod-ct.C2022Q4R9.cb_criteria as c
                JOIN
                    (
                        select
                            cast(cr.id as string) as id 
                        FROM
                            fc-aou-cdr-prod-ct.C2022Q4R9.cb_criteria as cr 
                        WHERE
                            concept_id IN (
                                1586134,1585855,1585710,43528895,40192389,1740639
                            ) 
                            AND domain_id = 'SURVEY'
                    ) a 
                        ON (
                            c.path like CONCAT('%',
                        a.id,
                        '.%')) 
                    WHERE
                        domain_id = 'SURVEY' 
                        AND type = 'PPI' 
                        AND subtype = 'QUESTION'
                    )
            )  
AND person_id IN ({})'''.format(', '.join(map(str, ids_list)))

surv_df = pd.read_gbq(survey_query, dialect="standard")
surv_df

In [ ]:
ses_query = '''
 SELECT
        observation.person_id,
        observation.observation_datetime,
        zip_code.zip3_as_string as zip_code,
        zip_code.fraction_assisted_income as assisted_income,
        zip_code.fraction_high_school_edu as high_school_education,
        zip_code.median_income,
        zip_code.fraction_no_health_ins as no_health_insurance,
        zip_code.fraction_poverty as poverty,
        zip_code.fraction_vacant_housing as vacant_housing,
        zip_code.deprivation_index,
        zip_code.acs as american_community_survey_year 
    FROM
        fc-aou-cdr-prod-ct.C2022Q4R9.zip3_ses_map as zip_code 
    JOIN
        fc-aou-cdr-prod-ct.C2022Q4R9.observation as observation 
            ON CAST(SUBSTR(observation.value_as_string,
        0,
        STRPOS(observation.value_as_string,
        '*') - 1) AS INT64) = zip_code.zip3 
    WHERE 
        observation_source_concept_id = 1585250 
        AND observation.value_as_string NOT LIKE 'Res%'
        AND person_id IN ({})'''.format(', '.join(map(str, ids_list)))

ses_df = pd.read_gbq(ses_query, dialect="standard")
ses_df

In [ ]:
# writing demographics file 

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = demo_df  


# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'demographics.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = surv_df  


# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'surveys.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = ses_df 


# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'ses.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
####examining distinct surveys  
#skeleton code below is used to read csv's that have been written to workspace storage  
 # Get ehr data from workspace bucket
name_of_file_in_bucket = 'surveys.csv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

# read in csv
surv_df = pd.read_csv(name_of_file_in_bucket)

In [ ]:
##########################
##question buckets
q_ducket = surv_df['survey'].unique()
print(q_ducket)
family_q = surv_df[surv_df['survey']=='Personal and Family Health History']['question'].unique()
basic_q = surv_df[surv_df['survey']=='The Basics']['question'].unique()
lifestyle_q = surv_df[surv_df['survey']=='Lifestyle']['question'].unique()
overall_health_q = surv_df[surv_df['survey']=='Overall Health']['question'].unique()
social_questions = surv_df[surv_df['survey']=='Social Determinants of Health']['question'].unique()
access_q = surv_df[surv_df['survey']=='Healthcare Access & Utilization']['question'].unique()

In [ ]:
print(lifestyle_q)
###include: Smoking: Smoke Frequency, Alchohol: Average Daily Drink Count, Past 3 Month Frequency*

In [ ]:
print(overall_health_q)
###Overall Health: Social Satisfaction, General Mental, General Physical

In [ ]:
print(access_q)
#Delayed Medical Care, Can't Afford Care 

In [ ]:
#print(family_q)
gi_search = ['colitis', 'bowel syndrome', 'Crohn', 'colon', 'gastro', 'intestine', 'rect']

trimmed_q = []
for s in family_q:
    for p in gi_search:
        if p in s and 'Including yourself' in s:
            trimmed_q.append(s)

In [ ]:
family_history = list(set(trimmed_q))
family_history

relevant_q = family_history+['Overall Health: Social Satisfaction', 'Overall Health: General Mental Health', 'Overall Health: General Physical Health','Delayed Medical Care', 'Can\'t Afford Care','food would run out', 'doctor or nurse act as if he or she thinks you are not smart', 'doctor or nurse is not listening', 'Smoking: Smoke Frequency', 'Alcohol: Average Daily Drink Count', 'Past 3 Month Use Frequency']
relevant_q
surv_df[surv_df['question'] == 'Including yourself, who in your family has had Crohn\'s disease? Select all that apply.']
'|'.join(relevant_q)


In [ ]:
import re
mask = surv_df['question'].str.contains(re.escape('Including yourself, who in your family has had Crohn\'s disease? Select all that apply.'), case=False)
#print(mask)

surv_df_test = surv_df[mask]

surv_df_test.head()

In [ ]:
####filtering relevant survey questions
mask = surv_df['question'].str.contains('|'.join(re.escape(q) for q in relevant_q), case=False)
#print(mask)

surv_df_fil = surv_df[mask]

surv_df_fil['question'].unique()

###dropping -Self answers from family history
mask = ~surv_df_fil['answer'].str.contains('- Self', case=False)
#print(mask)

surv_df_fil = surv_df_fil[mask]

surv_df_fil['question'].unique()


In [ ]:
trim_surv = surv_df_fil[['person_id', 'question', 'answer']]
print(len(trim_surv))
trim_surv = trim_surv.drop_duplicates(subset=['person_id', 'question'])
trim_surv = trim_surv.reset_index(drop=True)
surv_wide = trim_surv.pivot(index='person_id',columns='question',values='answer')

def extract_colon(cell):
    if isinstance(cell, str) and ':' in cell:
        return cell.split(':', 1)[-1].strip()
    else:
        return cell

surv_wide = surv_wide.applymap(extract_colon)



In [ ]:
def collapse_columns(row):
    cant_afford_columns = surv_wide.filter(like="Can't").columns
    if any(row[col] == 'Yes' for col in cant_afford_columns):
        return 'yes'
    elif all(pd.isna(row[col]) or row[col] == 'NA' for col in cant_afford_columns):
        return 'NA'
    else:
        return 'no'
    
def collapse_delay_columns(row):
    delayed_care_columns = surv_wide.filter(like="Delayed").columns
    if any(row[col] == 'Yes' for col in delayed_care_columns):
        return 'yes'
    elif all(pd.isna(row[col]) or row[col] == 'NA' for col in delayed_care_columns):
        return 'NA'
    else:
        return 'no'

In [ ]:
cant_afford_columns = surv_wide.filter(like="Can't").columns

surv_wide["Can't afford full medical care"] = surv_wide[cant_afford_columns].apply(collapse_columns, axis=1)
surv_wide = surv_wide.drop(columns=cant_afford_columns)

delayed_care_columns = surv_wide.filter(like="Delayed").columns

surv_wide["Delayed medical care"] = surv_wide[delayed_care_columns].apply(collapse_delay_columns, axis=1)
surv_wide = surv_wide.drop(columns=delayed_care_columns)

surv_wide["Can't afford full medical care"].unique()

In [ ]:
####examining distinct surveys to find relevant indicators 
#skeleton code below is used to read csv's that have been written to workspace storage  
 # Get ehr data from workspace bucket
name_of_file_in_bucket = 'demographics.csv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

# read in csv
demo_df = pd.read_csv(name_of_file_in_bucket)

In [ ]:
####examining distinct surveys to find relevant indicators 
#skeleton code below is used to read csv's that have been written to workspace storage  
 # Get ehr data from workspace bucket
name_of_file_in_bucket = 'ses.csv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

# read in csv
ses_df = pd.read_csv(name_of_file_in_bucket)

In [ ]:
ses_df.head()

ses_unneeded = ['observation_datetime', 'zip_code', 'high_school_education', 'vacant_housing', 'american_community_survey_year']

ses_df = ses_df.drop(columns = ses_unneeded)

In [ ]:
colitis['cancer_yes'] = colitis['age_at_crc_dx'].notna().astype(int)
colitis_final = colitis[['person_id','age_at_colitis_dx', 'uc1_crohns2', 'cancer_yes']]

sum(colitis_final['cancer_yes'])

In [ ]:
w_demo = pd.merge(colitis_final, demo_df.drop(columns=['date_of_birth']), on='person_id', how='left')
w_ses = pd.merge(w_demo, ses_df, on='person_id', how='left')
final = pd.merge(w_ses, surv_wide, on='person_id', how='left')
final.shape
#len(final['person_id'].unique())

for col in final.columns:
    print(col)

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = final 


# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'final_dataset.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr